In [3]:
import os

import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import datetime as dt
import math
from datetime import datetime

In [2]:
# Filepath to baseline daily river data
fpath_baseline_data = r"/home/jovyan/shared/common/oslofjord_modelling/MARTINI800v10_river_inputs/martini_rivers_v9_1990_2022_stage1data.nc"
netcdf_outfolder = r"/home/jovyan/shared/common/oslofjord_modelling/MARTINI800v10_river_inputs/river_scenarios"

# Set up & read in river metadata

In [3]:
if par in ['din', 'ton', 'totn']:
    species = 'N'
elif par in ['tdp', 'tpp', 'totp']:
    species = 'P'

riv_var_li_dict = {
    "N": [
        "river_NH4N",
        "river_NO3NO2N",
        "river_TOTN",
        "river_DON",
        "river_PON",
        "river_transport",
    ],
    "P": [
        "river_SRP",
        "river_TOTP",
        "river_DOP",
        "river_POP",
        "river_TIP",
        "river_transport",
    ],
}

derived_chemvar_dict = {
    "N": ['din', 'ton'],
    "P": ['tdp']
}

riv_var_li = riv_var_li_dict[species]
chem_var_li = riv_var_li.copy() + derived_chemvar_dict[species]
chem_var_li.remove('river_transport')

# Calculate start date (inclusive), end date (day after last day)
start_date = dt.datetime(start_year, 1, 1)
end_date = dt.datetime(end_year, 12, 31)
end_date += pd.Timedelta(days=1)

# River chemistry metadata
river_meta_df = pd.read_csv(real_riv_metadata, index_col=0, dtype={'Vassom':str})
# Limit to just Oslofjord rivers
river_meta_df = river_meta_df[river_meta_df.index.isin(oslofjord_riv_nos)]
# Add 'total' row for use later
river_meta_df.loc['Total', ['river_name', 'Regine', 'Regine_to_sea', 'Vassom']] = 'Total'
river_meta_df.query('real_river in @oslofjord_riv_nos')

,river_name,Outflow_lat,Outflow_lon,Regine,Regine_to_sea,Vassom,Vassom_area_land,Vassom_area_tot,Andre_MCA_area,Andre_area_q,Overestimate (%),Comment
real_river,,,,,,,,,,,,
6,Tista,59.119,11.370,001.A1,001.A1,001.,2495.0,2507.0,2507.0,1584.0,0.0,NaN
7,Glomma,59.206,10.953,002.A51,002.A11,002.,42446.0,43116.0,43116.0,41967.0,2.0,Monitoring point upstream of Sarpsborg RA. So ...
8,Mosseelva,59.439,10.662,003.A1,003.A1,003.,854.0,1052.0,1054.0,694.0,23.0,NaN
9,Hølenelva,59.523,10.690,004.A0,004.A0,004.,204.0,227.0,NaN,NaN,11.0,NaN
10,Årungen,59.720,10.728,005.3A,005.3A,005.,280.0,368.0,144.0,85.0,31.0,NaN
11,Akerselva,59.908,10.756,006.A10,006.A10,006.,392.0,415.0,415.0,307.0,6.0,NaN
12,Lysakerelva,59.914,10.640,007.A0,007.A0,007.,202.0,211.0,211.0,177.0,4.0,NaN
13,Sandvikselva,59.890,10.523,008.A11,008.A11,008.,279.0,311.0,311.0,226.0,11.0,NaN
14,Åros,59.704,10.519,009.A0,009.A0,009.,215.0,253.0,NaN,NaN,18.0,NaN


In [ ]:
def open_clean_netcdf (fpath):
    ds = xr.open_dataset(fpath)

    # Select just the variables of interest
    ds = ds[riv_var_li_dict[species]]

    # Convert to dataframe
    # N.B. loose units info then. m3/s for Q, ug/l for rest
    conc_df = ds.to_dataframe()
    ds.close()

    conc_df = conc_df.reset_index()  # Drop multiindex

    # # Add river name to df
    # conc_df['river_name'] = conc_df['real_river'].map(river_meta_df['river_name'])

    # Select just the Oslofjord rivers
    conc_df = conc_df[conc_df['real_river'].isin(oslofjord_riv_nos)]

    # Calculate DIN and TON
    conc_df['din'] = conc_df['river_NH4N'] + conc_df['river_NO3NO2N']
    # Assume particulate organic N = 0, and that this is more robust than TON = PON+DON
    conc_df['ton'] = conc_df['river_TOTN'] - conc_df['din']

    # Convert all numeric columns to float64 (chem cols were float32)
    conc_df[riv_var_li] = conc_df[riv_var_li].astype(float)

    # Round concentrations to 3 d.p. (N.B. only appropriate for ug/l)
    conc_df[chem_var_li] = conc_df[chem_var_li].round(3)

    # Truncate to start and end date
    conc_df = conc_df.query('@start_date <= river_time <= @end_date')
    
    conc_df.tail()

# Calculate daily & annual loads from the river data

## Daily loads

In [14]:
# Daily loads (kg/day)

# Make empty dataframe with just time & location info
daily_load_df = conc_df.drop(chem_var_li+['river_transport'], axis=1)

# Calculate load
for var in chem_var_li:
    # Units ug/l * m3/s * l/m3 * kg/ug * s/d = kg/d
    daily_load_df[var] = conc_df[var] * conc_df['river_transport'] * 10**3 * 10**-9 * 86400

chem_cols = daily_load_df.columns.drop(['river_time', 'real_river'])

daily_load_df.head()

,river_time,real_river,river_NH4N,river_NO3NO2N,river_TOTN,river_DON,river_PON,din,ton
327644,2013-01-01 12:00:00,6.0,99.304292,4136.529999,7003.108914,2526.572795,238.531498,4235.834311,2767.274610
327645,2013-01-01 12:00:00,7.0,692.954032,10022.521099,15417.079198,3971.521497,723.502474,10715.475200,4701.604138
327646,2013-01-01 12:00:00,8.0,199.130873,1214.673897,2189.321439,725.723523,49.342304,1413.806793,775.514680
327647,2013-01-01 12:00:00,9.0,68.907812,444.080250,735.532002,213.324874,9.136322,512.988079,222.544318
327648,2013-01-01 12:00:00,10.0,48.569385,674.590934,1159.604665,427.357783,9.005329,723.159846,436.444786


## Annual loads

In [15]:
# Annual loads (kg/year)
annual_load_df = daily_load_df.copy().set_index('river_time')
annual_load_df = annual_load_df.groupby('real_river').resample('YE').sum()

# Tidy
annual_load_df.drop('real_river', axis=1, inplace=True)
annual_load_df.reset_index(inplace=True)

# # Convert to tonnes (10**3 kg) per year
# annual_load_df[chem_cols] = annual_load_df[chem_cols]/1000

annual_load_df['year'] = annual_load_df['river_time'].dt.year
annual_load_df.drop('river_time', axis=1, inplace=True)

annual_load_df

,real_river,river_NH4N,river_NO3NO2N,river_TOTN,river_DON,river_PON,din,ton,year
0,6.0,12148.756964,4.696627e+05,8.548883e+05,348684.806476,24172.481743,4.818115e+05,373076.835932,2013
1,6.0,24691.520987,7.179538e+05,1.250792e+06,475092.293767,32759.244722,7.426454e+05,508146.259677,2014
2,6.0,12583.748320,6.601110e+05,1.144701e+06,440762.570758,30962.687071,6.726948e+05,472006.466481,2015
3,6.0,4797.933241,4.289918e+05,7.312321e+05,278736.468703,18537.616741,4.337896e+05,297442.459635,2016
4,6.0,13040.528094,5.503839e+05,9.138086e+05,329938.122509,20261.996188,5.634244e+05,350384.186715,2017
...,...,...,...,...,...,...,...,...,...
145,20.0,38339.002290,1.462085e+06,1.740374e+06,232419.643202,7462.601503,1.500424e+06,239950.040393,2018
146,20.0,91712.170890,1.822525e+06,2.073845e+06,143126.576566,16366.050079,1.914237e+06,159607.813147,2019
147,20.0,57215.250183,1.446076e+06,1.696930e+06,179414.366692,14098.026588,1.503292e+06,193638.226411,2020
148,20.0,84678.923005,1.018992e+06,1.320784e+06,208575.738556,8460.393411,1.103671e+06,217112.974208,2021


# Teotil3 summary data

File produced by James, summarise over whole vassdragsområder.

In [27]:
fpath = r"../data/teotil3_phase3_results_summary.csv"
t3_df = pd.read_csv(fpath)
t3_df.head()

,Scenario,Område,Kilde,Parameter,Verdi (tonn)
0,Baseline,001,Bakgrunn,DIN,106.293181
1,Baseline,001,Bakgrunn,SS,404.177361
2,Baseline,001,Bakgrunn,TDP,0.787351
3,Baseline,001,Bakgrunn,TOC,7407.268113
4,Baseline,001,Bakgrunn,TON,221.985402


In [29]:
slice_df = t3_df.query("Scenario == 'Baseline' and Område == '002' and Parameter == 'DIN'").copy()

total = slice_df['Verdi (tonn)'].sum(axis=0)
print(f"Total: {total.round(1)}")

slice_df

Total: 10636.9


,Scenario,Område,Kilde,Parameter,Verdi (tonn)
48,Baseline,002,Bakgrunn,DIN,1663.752438
56,Baseline,002,Bebygd,DIN,310.526000
64,Baseline,002,Industri,DIN,47.802970
72,Baseline,002,Jordbruk,DIN,6430.133930
80,Baseline,002,Kommunalt avløp,DIN,1932.804983
88,Baseline,002,Spredt avløp,DIN,251.922597
